<a href="https://colab.research.google.com/github/beedrumms/Exploring-Public-Responses-to-Government-COVID-19-Policies/blob/main/1)_ON_Combining_Scraped_Datasets_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re 
import string
import pandas as pd
import numpy as np
from google.colab import drive, files
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import nltk
from datetime import datetime

# Mounting google colab on drive 
from google.colab import drive, files
import os
drive.mount("/content/drive")

##Ontario scraped set had far less tweets then New York so it was scraped twice and sets were combined.

####Given how data is scraped using twint, datasets will be checked for duplicates and combined

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/covidsnscrape2.csv')
tweets_data = data[['Datetime', 'Text']]
tweets_data = tweets_data.drop(192335) # troublesome row
tweets_data = tweets_data.dropna()

In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/to_ford.csv')
data.shape, data2.shape
data2 = data2[['date', 'tweet']]

In [ ]:
listone = tweets_data['Text'].tolist()
listtwo = data2['tweet'].tolist()

In [ ]:
# Make sure everything is a string type
list1 = [] 
for t in listone:
  list1.append(str(t))

list2 = []
for t in listtwo:
  list2.append(str(t)) 

In [ ]:
tweets1 = []
for t in list1:
  tweets1.append(t.lower())

tweets_data['Text'] = tweets1

tweets2 = []
for t in list2:
  tweets2.append(t.lower())

data2['tweet'] = tweets2

In [ ]:
date_to_string = [] 
for t in tweets_data['Datetime']: 
  date_to_string.append(str(t))

import re 
convertdate = [] 
for t in date_to_string:
  convertdate.append(re.sub('\\s+[0-9][0-9]:[0-9][0-9]:[0-9][0-9][+][0-9][0-9][:][0-9][0-9]','', t))

tweets_data['Datetime'] = convertdate

In [ ]:
# Parsing dates as the two sets have different dates 
tweets_data['Datetime'] = pd.to_datetime(tweets_data['Datetime']) # converting to datetime
tweets_data['Datetime'] = tweets_data['Datetime'].dt.date # converting to just Year-Month-Day format 
tweets_data.Datetime.head(2), tweets_data.Datetime.tail(2)

(0    2021-05-31
 1    2021-05-31
 Name: Datetime, dtype: object, 308523    2020-03-01
 308524    2020-03-01
 Name: Datetime, dtype: object)

In [ ]:
data2['date'] = pd.to_datetime(data2['date']) # converting to datetime
data2['date'] = data2['date'].dt.date # converting to just Year-Month-Day format 
data2.date.head(2), data2.date.tail(2)
data2 = data2[ :248972] # two dates fall outside the desired date range 

In [ ]:
date1 = tweets_data['Datetime'].tolist()
date2 = data2['date'].tolist()
text1 = tweets_data['Text'].tolist()
text2 = data2['tweet'].tolist()

for i in date2: 
  date1.append(i)

for i in text2:
  text1.append(i)

len(date1), len(text1)

data_tuples = list(zip(date1, text1))

df = pd.DataFrame(data_tuples, columns = ['Datetime', 'Text']) 

df['Text'] = df['Text'].astype(str)

In [ ]:
df.drop_duplicates(subset=['Text'], inplace=True)
len(df['Text'])

421109

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/ONTARIOS_COVID19_TWEETS.csv')